In [33]:
# primer_tools.py
"""
Minimal helper functions for primer QC

Requires:
    pip install primer3-py
"""

import primer3  # Python bindings to the Primer3 C core
from pathlib import Path
''' Check to find overhangs and primers to work for a Gibson assembly of the two. The overhangs of the AF is not really debatable.
    This script is to find overhangsd in the HDR arms. Check file "name convention for an illustration.
'''

#read files
leftHDR = Path("leftHDR.txt").read_text(encoding="utf-8")
rightHDR = Path("rightHDR.txt").read_text(encoding="utf-8")
leftAF = Path("leftAF.txt").read_text(encoding="utf-8")
rightAF = Path("rightAF.txt").read_text(encoding="utf-8")

def analyze_primer(seq, mv=50.0, dv=1.5, dntp=0.0, dna=250.0):
    seq = seq.upper().replace(" ", "")
    hairpin = primer3.calcHairpin(seq)
    homodimer = primer3.calcHomodimer(seq)

    return {
        "sequence": seq,
        "tm": primer3.calcTm(seq, mv_conc=mv, dv_conc=dv,
                             dntp_conc=dntp, dna_conc=dna),
        "hairpin_dg": hairpin.dg,           # attribute, not key
        "homodimer_dg": homodimer.dg,
    }

def analyze_pair(fwd, rev, **kwargs):
    f_stats = analyze_primer(fwd, **kwargs)
    r_stats = analyze_primer(rev, **kwargs)
    hetero = primer3.calcHeterodimer(fwd, rev)

    return {
        "forward": f_stats,
        "reverse": r_stats,
        "heterodimer_dg": hetero.dg,
    }

def print_stats(stats):
    for side in ("forward", "reverse"):
        s = stats[side]
        print(f"{side:8}  Tm = {s['tm']:.1f} °C   "
              f"ΔGhairpin = {s['hairpin_dg']/1000:.1f} kcal/mol   "
              f"ΔGhomodimer = {s['homodimer_dg']/1000:.1f} kcal/mol ")

    print(f"Heterodimer ΔG = {stats['heterodimer_dg']/1000:.1f} kcal/mol")

def check_pair(fwd, rev):
    hairpin_treshold = -3000
    homodimer_treshold = -9000
    heterodimer_threshold = -9000

    stats = analyze_pair(fwd, rev)

    if stats['forward']['hairpin_dg'] < hairpin_treshold:
        return False
    if stats['forward']['homodimer_dg'] < homodimer_treshold:
        return False
    if stats['reverse']['hairpin_dg'] < hairpin_treshold:
        return False
    if stats['reverse']['homodimer_dg'] < homodimer_treshold:
        return False
    if stats['heterodimer_dg'] < heterodimer_threshold:
        return False
    
    return True

if __name__ == "__main__":
    FWD = "ACGTAGCTAGCCCCCTGATC"
    REV = "GATCGGGGGTaaaaaaCGT"
    stats = analyze_pair(FWD, REV)

    if check_pair(FWD, REV):
        print('Yaaaay')
    else:
        print('Nooooo')
    print_stats(stats)
    


Yaaaay
forward   Tm = 64.1 °C   ΔGhairpin = 0.0 kcal/mol   ΔGhomodimer = -6.4 kcal/mol 
reverse   Tm = 58.8 °C   ΔGhairpin = 0.0 kcal/mol   ΔGhomodimer = -2.4 kcal/mol 
Heterodimer ΔG = -7.2 kcal/mol


In [23]:
def get_snippet(
    seq: str,
    *,
    start: int,
    length: int,
    origin: str = "head",
) -> str:
    """
    Extract a subsequence of given length from `seq`.

    Parameters
    ----------
    seq : str
        Full sequence (upper/lower case OK).
    start : int
        Offset where to begin (0-based).  Interpretation depends on `origin`.
    length : int
        Number of characters to return.
    origin : {"head", "tail"}
        • "head" → start is counted from 5' end (index 0).  
        • "tail" → start is counted from 3' end (index len(seq)-1).

    Returns
    -------
    str
        Requested subsequence. Raises ValueError if requested range
        would go outside the sequence bounds.
    """
    seq = seq.upper()
    n = len(seq)

    if length <= 0:
        raise ValueError("length must be positive")

    if origin not in {"head", "tail"}:
        raise ValueError("origin must be 'head' or 'tail'")

    if origin == "head":
        i0 = start
    else:  # origin == "tail"
        i0 = n - start - length

    i1 = i0 + length

    if i0 < 0 or i1 > n:
        raise ValueError(
            f"requested range [{i0}:{i1}] lies outside sequence of length {n}"
        )

    return seq[i0:i1]

#check
a = "AGGTAGCATGACTGTTTAGTTTA"
b = "GGTAGCATGGGATACAGTATATT"
print(get_snippet(a,start=3,length=4,origin="tail"))

TAGT


In [36]:
'''Set parameters'''
#minimum length of overhang
min_overhang = 15
#maximum length of overhang
max_overhang = 26
#primer range
range_overhang = range(min_overhang, max_overhang)

#how many bases of the HDR arms can be skipped
latest_start_HDR = 300

#list for the good stuff
possible_primers = []

##### MAIN LOOP #####
#loops over possible left and right starts
for start_left in range(latest_start_HDR):
    for start_right in range(latest_start_HDR):

        #loops over possible lengths of overhangs
        for oh_leftHDR in range_overhang:
            for oh_rightHDR in range_overhang:
                for oh_leftAF in range_overhang:
                    for oh_rightAF in range_overhang:

                        #define sequences (see file "namin convention.pdf" for orientation)
                        HDRfw = get_snippet(rightAF, start=0, length=oh_rightAF, origin="tail") + get_snippet(leftHDR, start=start_left, length=oh_leftHDR, origin="head")
                        HDRrv = get_snippet(rightHDR, start=start_right, length=oh_rightHDR, origin="tail") + get_snippet(leftAF, start=0, length=oh_rightAF, origin="head")

                        if check_pair(HDRfw, HDRrv):
                            if check_pair
                            print("I GOT A PAIR!")
                            possible_primers.append((HDRfw,HDRrv))
                            print_stats(analyze_pair(HDRfw, HDRrv))
                       

                        



/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:135: UserWarning: Function deprecated please use "calc_hairpin" instead
  return THERMO_ANALYSIS.calcHairpin(seq, output_structure).check_exc()
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:217: UserWarning: Function deprecated please use "calc_homodimer" instead
  return THERMO_ANALYSIS.calcHomodimer(seq, output_structure).check_exc()
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:489: UserWarning: Function deprecated please use "calc_tm" instead
  return THERMO_ANALYSIS.calcTm(seq)
/Users/jonfreeman/.pyenv/versions/3.10.13/envs/jupyter/lib/python3.10/site-packages/primer3/bindings.py:305: UserWarning: Function deprecated please use "calc_heterodimer" instead
  return THERMO_ANALYSIS.calcHeterodimer(


I GOT A PAIR!
forward   Tm = 75.6 °C   ΔGhairpin = -2.5 kcal/mol   ΔGhomodimer = -7.8 kcal/mol 
reverse   Tm = 69.7 °C   ΔGhairpin = 0.1 kcal/mol   ΔGhomodimer = -6.7 kcal/mol 
Heterodimer ΔG = -4.6 kcal/mol
I GOT A PAIR!
forward   Tm = 76.3 °C   ΔGhairpin = -2.5 kcal/mol   ΔGhomodimer = -7.8 kcal/mol 
reverse   Tm = 70.5 °C   ΔGhairpin = 0.1 kcal/mol   ΔGhomodimer = -6.7 kcal/mol 
Heterodimer ΔG = -4.6 kcal/mol
I GOT A PAIR!
forward   Tm = 76.2 °C   ΔGhairpin = -2.5 kcal/mol   ΔGhomodimer = -7.8 kcal/mol 
reverse   Tm = 70.8 °C   ΔGhairpin = 0.1 kcal/mol   ΔGhomodimer = -6.7 kcal/mol 
Heterodimer ΔG = -4.9 kcal/mol
I GOT A PAIR!
forward   Tm = 76.4 °C   ΔGhairpin = -2.5 kcal/mol   ΔGhomodimer = -7.8 kcal/mol 
reverse   Tm = 71.5 °C   ΔGhairpin = 0.1 kcal/mol   ΔGhomodimer = -6.7 kcal/mol 
Heterodimer ΔG = -5.5 kcal/mol
I GOT A PAIR!
forward   Tm = 77.5 °C   ΔGhairpin = -2.5 kcal/mol   ΔGhomodimer = -7.8 kcal/mol 
reverse   Tm = 71.7 °C   ΔGhairpin = 0.1 kcal/mol   ΔGhomodimer = -6.7 k

KeyboardInterrupt: 